In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

In [168]:
hr12_arrival = [0] * 179
hr12_arrival.append(1)
hr12_arrival = hr12_arrival * 40

hr34_arrival = [0] * 71
hr34_arrival.append(1)
hr34_arrival = hr34_arrival * 100

hr5_arrival = [0] * 50
hr5_arrival.append(1)
hr5_arrival = hr5_arrival * 70

In [120]:
#hr12_arrival = [1]
#hr12_arrival.extend([0] * 179)
#hr12_arrival = hr12_arrival * 40

#hr34_arrival = [1]
#hr34_arrival.extend([0] * 71)
#hr34_arrival = hr34_arrival * 100

#hr5_arrival = [1]
#hr5_arrival.extend([0] * 50)
#hr5_arrival = hr5_arrival * 70

In [169]:


time = pd.date_range(start='2024-02-28 00:00:00', end='2024-02-28 5:00:00', freq='s')

arrivals = hr12_arrival + hr34_arrival + hr5_arrival
extra_a = [0] * (len(time) - len(arrivals))
arrivals = arrivals + extra_a

hr12_output = [0] * 134
hr12_output.append(1)
hr12_output.extend([0] * 45)
hr12_output = hr12_output * 40

hr3_start = [0] * 134
hr3_start.append(1)

hr345_output = [0] * 74
hr345_output.append(1)
hr345_output = hr345_output * 142

output = hr12_output + hr3_start + hr345_output
extra_o = [0] * (len(time) - len(output))
output = output + extra_o

df = pd.DataFrame({'time': time, 'arrivals': arrivals, 'output': output})
df['buildup'] = df.arrivals.cumsum() - df.output.cumsum()
print(df.arrivals.sum(), df.output.sum(), df.buildup.max())


210 183 28


In [170]:
output_rate = df.output.sum() / ((len(df.output)-1)/3600)
output_rate12 = df.loc[df.time < pd.to_datetime('2024-02-28 02:00:00'),'output'].sum() / ((len(df.loc[df.time < pd.to_datetime('2024-02-28 02:00:00'),'output'])-1)/3600)
output_rate345 = df.loc[df.time >= pd.to_datetime('2024-02-28 02:00:00'),'output'].sum() / ((len(df.loc[df.time >= pd.to_datetime('2024-02-28 02:00:00'),'output'])-1)/3600)
print(f'The average throughput or output rate was {output_rate12:.2f} customers per hour in \nthe first two hours, {output_rate345:.2f} in the last three hours, and {output_rate:.2f} overall.')

The average throughput or output rate was 20.00 customers per hour in 
the first two hours, 47.67 in the last three hours, and 36.60 overall.


In [173]:
dfmin = df.groupby(pd.Grouper(key='time', freq='min')).agg({'arrivals': 'sum','output': 'sum'}).reset_index()
dfmin['buildup'] = np.cumsum(dfmin['arrivals']) - np.cumsum(dfmin['output'])

#plt.figure(figsize=(12, 6))
#plt.bar(dfmin.time, dfmin.buildup, width=0.001)

time_format = DateFormatter('%H:%M')  
#plt.gca().xaxis.set_major_formatter(time_format)

#plt.xlabel('Time')
#plt.ylabel('Customers in Line')
#plt.title('Build Up of Customers in Line')
#plt.show()

print(f"Total arrivals: {dfmin.arrivals.sum()}, total output: {dfmin.output.sum()}, final build up: {dfmin.buildup.max()}")
print(f"Total time waiting in line: {dfmin.buildup.sum()} minutes")

total_ctime = dfmin.buildup.sum() # in customer-minutes
print(dfmin.loc[dfmin.buildup > 0, 'buildup'].mean(), dfmin.buildup.mean())
cust_that_waited = dfmin.loc[dfmin.buildup > 0, 'arrivals'].sum() - 1
minutes_with_waiting = dfmin.loc[dfmin.buildup > 0, 'time'].count() - 1

print(f"Customers that waited in line: {cust_that_waited}, minutes with waiting: {minutes_with_waiting}, average customers waiting: {total_ctime/minutes_with_waiting:.2f}, average wait time per customer: {total_ctime/cust_that_waited:.2f} minutes - {total_ctime/dfmin.arrivals.sum():.2f} minutes per customer")

Total arrivals: 210, total output: 183, final build up: 27
Total time waiting in line: 1336 minutes
7.4222222222222225 4.438538205980066
Customers that waited in line: 169, minutes with waiting: 179, average customers waiting: 7.46, average wait time per customer: 7.91 minutes - 6.36 minutes per customer


In [174]:
time = pd.date_range(start='2024-02-28 00:00:00', end='2024-02-28 6:00:00', freq='s')

arrivals = hr12_arrival + hr34_arrival + hr5_arrival
extra_a = [0] * (len(time) - len(arrivals))
arrivals = arrivals + extra_a

hr12_output = [0] * 134
hr12_output.append(1)
hr12_output.extend([0] * 45)
hr12_output = hr12_output * 40

hr3_start = [0] * 134
hr3_start.append(1)

hr345_output = [0] * 74
hr345_output.append(1)
hr345_output = hr345_output * 190

output = hr12_output + hr3_start + hr345_output
extra_o = [0] * (len(time) - len(output))
output = output + extra_o

dfext = pd.DataFrame({'time': time, 'arrivals': arrivals, 'output': output})
dfext['buildup'] = dfext.arrivals.cumsum() - dfext.output.cumsum()

In [175]:
dfxmin = dfext.groupby(pd.Grouper(key='time', freq='min')).agg({'arrivals': 'sum','output': 'sum'}).reset_index()
dfxmin['buildup'] = np.cumsum(dfxmin['arrivals']) - np.cumsum(dfxmin['output'])

print(f"Total arrivals: {dfxmin.arrivals.sum()}, total output: {dfxmin.output.sum()}, final build up: {dfxmin.buildup.max()}")
print(f"Total time waiting in line: {dfxmin.loc[dfxmin.time < pd.to_datetime('2024-02-28 05:33:00'), 'buildup'].sum()} customer-minutes")

total_ctime = dfxmin.loc[dfxmin.buildup > 0, 'buildup'].sum() # in customer-minutes
print(total_ctime)
#print(dfxmin.loc[dfxmin.buildup > 0, 'buildup'].mean(), dfxmin.buildup.mean())
cust_that_waited = dfxmin.loc[dfxmin.buildup > 0, 'arrivals'].sum() - 1
minutes_with_waiting = dfxmin.loc[dfxmin.buildup > 0, 'time'].count() - 1

print(f"Customers that waited in line: {cust_that_waited}, minutes with waiting: {minutes_with_waiting}, average customers waiting: {total_ctime/minutes_with_waiting:.2f}, average wait time per customer: {total_ctime/cust_that_waited:.2f} minutes")


output_rate = dfxmin.output.sum() / ((len(dfxmin.output)-1)/60)
output_rate6 = dfxmin.loc[(dfxmin.buildup > 0) & (dfxmin.time > pd.to_datetime('2024-02-28 05:00:00')),'output'].sum() / ((len(dfxmin.loc[dfxmin.time > pd.to_datetime('2024-02-28 05:00:00'),'output'])-1)/60)
print(f'The average throughput or output rate was {output_rate:.2f} customers per hour , and in the last hour {output_rate6:.2f} .')



Total arrivals: 210, total output: 231, final build up: 27
Total time waiting in line: 1758 customer-minutes
1758
Customers that waited in line: 169, minutes with waiting: 211, average customers waiting: 8.33, average wait time per customer: 10.40 minutes
The average throughput or output rate was 38.50 customers per hour , and in the last hour 25.42 .


In [ ]:
x = dfxmin.loc[dfxmin.buildup > 0, 'time']
y = dfxmin.loc[dfxmin.buildup > 0, 'buildup']

x = dfxmin.loc[(dfxmin.time < pd.to_datetime('2024-02-28 05:00:00')) | (dfxmin.buildup > 0), 'time']
y = dfxmin.loc[(dfxmin.time < pd.to_datetime('2024-02-28 05:00:00')) | (dfxmin.buildup > 0), 'buildup']

plt.figure(figsize=(12, 6))
plt.plot(x, y)

time_format = DateFormatter('%H:%M')  
plt.gca().xaxis.set_major_formatter(time_format)

plt.xlabel('Time')
plt.ylabel('Customers in Line')
plt.title('Build Up of Customers in Line')
plt.show()

#dfxmin.to_csv('data/phobuildup.csv', index=False)

In [41]:
# I = F * T

inv1 = 1  #patient waiting
flow1 = inv1 / tput1 #hours
tput1 = 10  #patients per hour

print(f"Inventory: {inv1} patient waiting, flow: {flow1} hours or {flow1*60} minutes, throughput: {tput1} patients per hour")

Inventory: 1 patient waiting, flow: 0.1 hours or 6.0 minutes, throughput: 10 patients per hour


In [42]:
flow2 = 0.25 #hours
tput2 = 10  #patients per hour
inv2 = flow2 * tput2  #patient waiting

print(f"Inventory: {inv2} patients checking in, flow: {flow2} hours or {flow2*60} minutes, throughput: {tput2} patients per hour")

Inventory: 2.5 patients checking in, flow: 0.25 hours or 15.0 minutes, throughput: 10 patients per hour


In [43]:
flow3 = 0.25 #hours *new time*
tput3 = 10  #patients per hour
inv3 = flow3 * tput3  #patients waiting

print(f"Inventory: {inv3} patients waiting, flow: {flow3} hours or {flow3*60} minutes, throughput: {tput3} patients per hour")

Inventory: 2.5 patients waiting, flow: 0.25 hours or 15.0 minutes, throughput: 10 patients per hour


In [46]:
flow4it = 3 #hours
flow4iw = 2 #hours
flow4i = flow4it + flow4iw
tput4i = 2  #patients per hour
inv4i = flow4i * tput4i  #patient waiting
inv4it = flow4it * tput4i  #patients in treatment
inv4iw = flow4iw * tput4i  #patients waiting for ward

flow4d = 3 #hours
tput4d = 8  #patients per hour
inv4d = flow4d * tput4d  #patient in treatment

inv4 = inv4i + inv4d
tput4 = tput4i + tput4d
flow4 = (inv4/tput4) #hours
#flow4 = (0.8 * 3) + (0.2 * 5)

print(f"Inventory: {inv4} patients in treatment, flow: {flow4} hours, throughput: {tput4} patients per hour")
print(f"For discharge patients, average inventory is {inv4d} patients, flow is {flow4d} hours, throughput is {tput4d} patients per hour")
print(f"For internal ward patients in treatment, average inventory is {inv4it} patients, flow is {flow4it} hours, throughput is {tput4i} patients per hour")
print(f"For internal ward patients waiting, average inventory is {inv4iw} patients, flow is {flow4iw} hours, throughput is {tput4i} patients per hour")
print(f"For internal ward patients, average inventory is {inv4i} patients, flow is {flow4i} hours, throughput is {tput4i} patients per hour")


Inventory: 34 patients in treatment, flow: 3.4 hours, throughput: 10 patients per hour
For discharge patients, average inventory is 24 patients, flow is 3 hours, throughput is 8 patients per hour
For internal ward patients in treatment, average inventory is 6 patients, flow is 3 hours, throughput is 2 patients per hour
For internal ward patients waiting, average inventory is 4 patients, flow is 2 hours, throughput is 2 patients per hour
For internal ward patients, average inventory is 10 patients, flow is 5 hours, throughput is 2 patients per hour


In [47]:
flow5 = 0.5  #hours
tput5 = (0.8 * 10)  #patients per hour
inv5 = flow5 * tput5  #patient waiting

print(f"Inventory: {inv5} patients being discharged, flow: {flow5} hours or {flow5*60} minutes, throughput: {tput5} patients per hour")

Inventory: 4.0 patients being discharged, flow: 0.5 hours or 30.0 minutes, throughput: 8.0 patients per hour


In [20]:
flow5i = 3.5 #hours
tput5i = (0.2 * 10)  #patients per hour
inv5i = flow5i * tput5i  #patient waiting

print(f"Inventory: {inv5i} patients waiting for transfer, flow: {flow5i} hours or {flow5i*60} minutes, throughput: {tput5i} patients per hour")

Inventory: 7.0 patients waiting for transfer, flow: 3.5 hours or 210.0 minutes, throughput: 2.0 patients per hour


In [48]:
discharge = flow1 + .25 + flow3 + 3 + 0.5
internal = flow1 + .25 + flow3 + 3 + 2
avflow = (0.8 * discharge) + (0.2 * internal)
total_avg_flow = flow1 + .25 + flow3 + 3 + (.2 * 2) + (.8 * 0.5)
print(f"Total average flow: {total_avg_flow} hours or {total_avg_flow*60} minutes, average flow other calc: {avflow} hours")
print(f"Discharge flow time: {discharge} hours, internal flow time: {internal} hours")
print(f"Flow for step 1 is {flow1} hours and for step 3 is {flow3} hours")

Total average flow: 4.4 hours or 264.0 minutes, average flow other calc: 4.3999999999999995 hours
Discharge flow time: 4.1 hours, internal flow time: 5.6 hours
Flow for step 1 is 0.1 hours and for step 3 is 0.25 hours
